In [6]:
import pandas as pd
import requests
import re
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time


In [2]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'}
datafilms=pd.read_csv('datafilmsParental.csv', index_col=0)
datafilms.insert(12, 'visaCNC', [None]*len(datafilms))

In [3]:
datafilms

,id,title,type,averageRating,numVotes,releaseYear,idjw,NbDiff,ClassifImdb,Diffuseur,...,Genre_horror,Genre_music,Genre_musical,Genre_mystery,Genre_romance,Genre_sci-fi,Genre_sport,Genre_thriller,Genre_war,Genre_western
0,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
1,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://play.max.com/show/9b4dacba-2f80-4272-a...,...,0,0,0,0,0,0,0,0,0,0
2,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://www.mycanal.fr/cinema/les-evades/h/404...,...,0,0,0,0,0,0,0,0,0,0
3,tt0468569,The Dark Knight,movie,9.0,3021134.0,2008.0,tm56184,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
4,tt0468569,The Dark Knight,movie,9.0,3021134.0,2008.0,tm56184,2.0,tous publics,https://play.max.com/show/52217243-a137-45d6-9...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,tt9663764,Aquaman and the Lost Kingdom,movie,5.6,106723.0,2023.0,tm460935,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
2720,tt0054047,The Magnificent Seven,movie,7.7,106685.0,1960.0,tm205009,2.0,tous publics,https://www.canalplus.com/cinema/les-sept-merc...,...,0,0,0,0,0,0,0,0,0,0
2721,tt0054047,The Magnificent Seven,movie,7.7,106685.0,1960.0,tm205009,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
2722,tt0318649,Sahara,movie,6.1,106564.0,2005.0,tm172656,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0


In [4]:

def visaCNC(url, annee):
    options = Options()
    options.add_argument("--headless")
    driver=webdriver.Chrome(options=options)
    time.sleep(0.00000001)
    driver.get(url)
    html= driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    html=soup.find_all('div', class_='meta', )
    page=""
    for k in html[:9]:
        page+=str(k)
    liens=re.findall(r'/film/fichefilm_gen_cfilm=(.+)\.html.+\n+.+\n.+\n.+\n<span class="date">.+(\d{4})</span>', page)
    codeurl=''
    for k in liens:
        if k[1]==annee:
            codeurl=k[0]
            break
    site=requests.get(f'https://www.allocine.fr/film/fichefilm_gen_cfilm={codeurl}.html', headers=headers)
    visa=re.findall(r'<span class="that">(.+)</span>', site.text)
    driver.quit()
    
    return visa[-1]
        
    


In [5]:
url=f'https://www.allocine.fr/rechercher/movie/?q={urllib.parse.quote('The Godfather')}'
print(visaCNC(url, '1972'))

39917


In [6]:
nomF=''
eta=0
prob=set()
for k in datafilms.index:
    if datafilms.loc[k, "title"]!=nomF:
        res=''
        eta+=1
        nomF=datafilms.loc[k, "title"]
        print(f"{nomF}      {eta}/1500")
        try:
            res=visaCNC(f"https://www.allocine.fr/rechercher/movie/?q={urllib.parse.quote(nomF)}", str(int(datafilms.loc[k, "releaseYear"])))
            datafilms.loc[k, 'visaCNC']=res
        except Exception as e:
            prob.add((nomF, e))
    else:
        if res:
            datafilms.loc[k, 'visaCNC']=res
            
    
print(prob)
print(len(prob))

The Shawshank Redemption      1/1500
The Dark Knight      2/1500
Inception      3/1500
Fight Club      4/1500
Forrest Gump      5/1500
Interstellar      6/1500
Pulp Fiction      7/1500
The Matrix      8/1500
The Godfather      9/1500
The Lord of the Rings: The Fellowship of the Ring      10/1500
The Lord of the Rings: The Return of the King      11/1500
The Dark Knight Rises      12/1500
The Lord of the Rings: The Two Towers      13/1500
Django Unchained      14/1500
Gladiator      15/1500
The Wolf of Wall Street      16/1500
Inglourious Basterds      17/1500
Batman Begins      18/1500
The Silence of the Lambs      19/1500
Joker      20/1500
Saving Private Ryan      21/1500
Shutter Island      22/1500
The Prestige      23/1500
Star Wars: Episode IV - A New Hope      24/1500
The Avengers      25/1500
The Departed      26/1500
The Green Mile      27/1500
Star Wars: Episode V - The Empire Strikes Back      28/1500
Avatar      29/1500
The Godfather Part II      30/1500
Back to the Future  

In [7]:
datafilms=pd.read_csv('datafilmsCNC.csv', index_col=0)

,id,title,type,averageRating,numVotes,releaseYear,idjw,NbDiff,ClassifImdb,Diffuseur,...,Genre_horror,Genre_music,Genre_musical,Genre_mystery,Genre_romance,Genre_sci-fi,Genre_sport,Genre_thriller,Genre_war,Genre_western
0,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
1,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://play.max.com/show/9b4dacba-2f80-4272-a...,...,0,0,0,0,0,0,0,0,0,0
2,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://www.mycanal.fr/cinema/les-evades/h/404...,...,0,0,0,0,0,0,0,0,0,0
27,tt1853728,Django Unchained,movie,8.5,1791607.0,2012.0,tm41792,1.0,12,https://www.netflix.com/title/70230640,...,0,0,0,0,0,0,0,0,0,1
53,tt0120689,The Green Mile,movie,8.6,1485848.0,1999.0,tm54285,2.0,12,https://play.max.com/show/d438e592-c8ae-4f79-a...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,tt9362930,Blue Beetle,movie,5.9,108693.0,2023.0,tm452641,3.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,1,0,0,0,0
2702,tt0156887,Perfect Blue,movie,8.0,107939.0,1997.0,tm190768,1.0,12,https://www.crunchyroll.com/watch/G0DUNJP9Q,...,0,0,0,0,0,0,0,0,0,0
2710,tt0098206,Road House,movie,6.7,107267.0,1989.0,tm1425500,1.0,tous publics avec avertissement,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,1,0,0
2720,tt0054047,The Magnificent Seven,movie,7.7,106685.0,1960.0,tm205009,2.0,tous publics,https://www.canalplus.com/cinema/les-sept-merc...,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def siteCNC(nomF, annee):
    site=pd.read_html(f'https://www.cnc.fr/professionnels/visas-et-classification?_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_formDate=1750263819092&p_p_id=RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_executeRechercheMultiCriteres=executeRechercheMultiCriteres&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_titre={urllib.parse.quote(nomF)}&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_realisateur=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_mention=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_dateDebut=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_dateFin=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_metrage=LM',
                      attrs={'class':"table table-striped table-responsive-lg sortable"})[0]
    if len(site)==1:
        return site.loc[0, 'Visa']
    for k in site.index:
        if site.loc[k, 'Date de sortie'][-4:]==annee:
            return int(site.loc[k, 'Visa'])
            

In [34]:
print(siteCNC('shawshank redemption', '1994'))

87498


In [32]:
datafilms=pd.read_csv('resfilmsCNC.csv', index_col=0)

In [33]:
len(datafilms[datafilms['visaCNC']=='-'])

318

In [34]:
nfilm=''
prob=[]
eta=0
listnone=[]
for h in datafilms.loc[datafilms['visaCNC']=='-'].index:
        if nfilm!=datafilms.loc[h, 'title']:
            nfilm=datafilms.loc[h, 'title']
            eta+=1
            print(f"{nfilm}     {eta} / 318")
            try:
                res=siteCNC(nfilm, str(int(datafilms.loc[h, 'releaseYear'])))
                if res!=None:
                    print(res)
                    datafilms.loc[h, 'visaCNC']=res
                else:
                    listnone.append(nfilm)
                    print(f'^^^^^^^^^^^^^MANUEL {datafilms.loc[h, 'releaseYear']}^^^^^^^^^^^^^^^^^^')
            except Exception as e:
                prob.append((nfilm, e))
                print(f"----------------------{nfilm, datafilms.loc[h, 'releaseYear']}-------------------------")
                
        else:
            if res:
                datafilms.loc[h, 'visaCNC']=res
                
print(prob)
print(len(prob))

print(listnone)
print(len(listnone))
        

Saving Private Ryan     1 / 318
95271
Alien     2 / 318
51085
Die Hard     3 / 318
68206
Spider-Man: No Way Home     4 / 318
156030
Ant-Man     5 / 318
142583
Home Alone     6 / 318
75021
The Notebook     7 / 318
111049
Zombieland     8 / 318
124272
Don't Look Up     9 / 318
----------------------("Don't Look Up", np.float64(2021.0))-------------------------
Watchmen     10 / 318
122432
Back to the Future Part II     11 / 318
73857
500 Days of Summer     12 / 318
123536
Mr. & Mrs. Smith     13 / 318
113283
X-Men Origins: Wolverine     14 / 318
----------------------('X-Men Origins: Wolverine', np.float64(2009.0))-------------------------
War of the Worlds     15 / 318
112935
In Bruges     16 / 318
120636
Glass Onion     17 / 318
----------------------('Glass Onion', np.float64(2022.0))-------------------------
Zack Snyder's Justice League     18 / 318
----------------------("Zack Snyder's Justice League", np.float64(2021.0))-------------------------
Transformers: Revenge of the Fallen 

In [35]:
datafilms.to_csv('datafilmsCNC25.csv')

In [37]:
datafilms[datafilms['visaCNC']=='-']

,id,title,type,averageRating,numVotes,releaseYear,idjw,NbDiff,ClassifImdb,Diffuseur,...,Genre_horror,Genre_music,Genre_musical,Genre_mystery,Genre_romance,Genre_sci-fi,Genre_sport,Genre_thriller,Genre_war,Genre_western
385,tt11286314,Don't Look Up,movie,7.2,640211.0,2021.0,tm856378,1.0,18,https://www.netflix.com/title/81252357,...,0,0,0,0,0,1,0,0,0,0
489,tt0458525,X-Men Origins: Wolverine,movie,6.5,554643.0,2009.0,tm154915,1.0,tous publics,https://disneyplus.bn5x.net/c/1206980/705874/9...,...,0,0,0,0,0,1,0,0,0,0
622,tt11564570,Glass Onion,movie,7.1,476628.0,2022.0,tm915749,1.0,13::(self-applied),https://www.netflix.com/title/81458416,...,0,0,0,0,0,0,0,0,0,0
664,tt12361974,Zack Snyder's Justice League,movie,7.9,455985.0,2021.0,tm893885,3.0,12::(self-applied),https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
695,tt1055369,Transformers: Revenge of the Fallen,movie,6.0,441836.0,2009.0,tm138894,4.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,tt13452446,Damsel,movie,6.1,110162.0,2024.0,tm1187584,1.0,NaN,https://www.netflix.com/title/80991090,...,0,0,0,0,0,0,0,0,0,0
2681,tt15097216,Jai Bhim,movie,8.6,225748.0,2021.0,tm1108724,1.0,16::(self applied),https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
2696,tt0248667,Ali,movie,6.7,108168.0,2001.0,tm57131,3.0,tous publics,https://play.max.com/show/0384b2a3-74bf-47fc-a...,...,0,0,0,0,0,0,1,0,0,0
2697,tt0248667,Ali,movie,6.7,108168.0,2001.0,tm57131,3.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,1,0,0,0


In [12]:
df2=pd.read_csv('datafilmsCNC.csv',index_col=0)

In [13]:
df1=pd.read_csv('datafilmsCNC2.csv', index_col=0)

In [14]:
df=df1.combine_first(df2)

In [46]:
datafilms=pd.read_csv('datafilmsCNC25.csv', index_col=0)

In [47]:
datafilms

,id,title,type,averageRating,numVotes,releaseYear,idjw,NbDiff,ClassifImdb,Diffuseur,...,Genre_horror,Genre_music,Genre_musical,Genre_mystery,Genre_romance,Genre_sci-fi,Genre_sport,Genre_thriller,Genre_war,Genre_western
0,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
1,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://play.max.com/show/9b4dacba-2f80-4272-a...,...,0,0,0,0,0,0,0,0,0,0
2,tt0111161,The Shawshank Redemption,movie,9.3,3044564.0,1994.0,tm119206,3.0,tous publics avec avertissement,https://www.mycanal.fr/cinema/les-evades/h/404...,...,0,0,0,0,0,0,0,0,0,0
3,tt0468569,The Dark Knight,movie,9.0,3021134.0,2008.0,tm56184,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
4,tt0468569,The Dark Knight,movie,9.0,3021134.0,2008.0,tm56184,2.0,tous publics,https://play.max.com/show/52217243-a137-45d6-9...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,tt9663764,Aquaman and the Lost Kingdom,movie,5.6,106723.0,2023.0,tm460935,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
2720,tt0054047,The Magnificent Seven,movie,7.7,106685.0,1960.0,tm205009,2.0,tous publics,https://www.canalplus.com/cinema/les-sept-merc...,...,0,0,0,0,0,0,0,0,0,0
2721,tt0054047,The Magnificent Seven,movie,7.7,106685.0,1960.0,tm205009,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0
2722,tt0318649,Sahara,movie,6.1,106564.0,2005.0,tm172656,2.0,tous publics,https://app.primevideo.com/detail?gti=amzn1.dv...,...,0,0,0,0,0,0,0,0,0,0


In [48]:
nomf=''
for i in datafilms.index:
    if datafilms.loc[i, 'visaCNC']=='-' and datafilms.loc[i, 'NbDiff']>1:
        print(datafilms.loc[i, 'title'])

Prey
Prey
EuroTrip
EuroTrip
EuroTrip
X
X
Mortal Kombat
Mortal Kombat
Army of the Dead
CODA
CODA
Paris, Texas
Greyhound


Webscraping Age

In [2]:
datafilms=pd.read_csv('testagecnc.csv', index_col=0)

In [10]:
idf=''
eta=0
prob=[]
for k in datafilms.index:
    if idf!=datafilms.loc[k, 'id']:
        idf=datafilms.loc[k, 'id']
        if pd.isna(datafilms.loc[k, 'classifCNC']):
            eta+=1
            print(f"{datafilms.loc[k, 'title']}      {eta} / 110 ")
            try:
                datarech=pd.read_html(f'https://www.cnc.fr/professionnels/visas-et-classification?_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_formDate=1750428563837&p_p_id=RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_executeRechercheMultiCriteres=executeRechercheMultiCriteres&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_titre={urllib.parse.quote(datafilms.loc[k, 'title'])}&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_realisateur=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_mention=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_dateDebut=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_dateFin=&_RechercheVisaClassificationControlleur_INSTANCE_bFazQlIweBh2_metrage=LM', attrs={"class":"table table-striped table-responsive-lg sortable"})[0]
                for h in datarech.index:
                    print(datarech.loc[h, 'Visa'], datafilms.loc[k, 'visaCNC'])
                    if str(datarech.loc[h, 'Visa']).replace(' ','')==str(datafilms.loc[k, 'visaCNC']).replace(' ',''):
                        print('oui')
                        datafilms.loc[k, 'classifCNC']=datarech.loc[h, 'Mentions']
            except Exception as e:
                prob.append((datafilms.loc[k, 'title']))

Joker      1 / 110 
151541 151541
oui
141099 151541
162947 151541
20536 151541
33197 151541
Back to the Future      2 / 110 
60856 60261
72299 60261
73857 60261
X-Men: Days of Future Past      3 / 110 
12 Years a Slave      4 / 110 
138661 138661
oui
Kick-Ass      5 / 110 
126096 126096
oui
137730 126096
The Conjuring      6 / 110 


KeyboardInterrupt: 